# Django Forms

In [ ]:
python manage.py startapp forms_test_recap

### src/forms_test_recap/forms.py

In [ ]:
from django import forms

class SearchForm(forms.Form):
    q = forms.CharField()

### Mostrar el form en una vista

### src/forms_test_recap/views.py

In [ ]:
from django.shortcuts import render

from .forms import SearchForm

def home(request):
    form = RecapForm()
    return render(request, "forms.html", {"form":form})

### src/config/urls.py

In [ ]:
from django.conf import settings
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce/", include("ecommerce.urls")),
    path("products/", include("products.urls")), 
    path("forms/", include("forms_test.urls")), 
    path("forms-recap/", include("forms_test_recap.urls")), # <-------
    path("admin/", admin.site.urls),
]
if not settings.TESTING:
    urlpatterns = [
        *urlpatterns,
        path("__debug__/", include("debug_toolbar.urls")),
    ]


### src/forms_test_recap/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path

from .views import home

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", home),
]

### src/config/settings.py

In [ ]:
# Application definitions
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "products.apps.ProductsConfig",
    "ecommerce.apps.EcommerceConfig",
    "base.apps.BaseConfig", 
    "forms_test.apps.FormsTestConfig",
    "forms_test_recap.apps.FormsTestRecapConfig", # <-------
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

## Validación en campos de un Form

### src/forms_test_recap/forms.py

In [ ]:
from django import forms

class RecapForm(forms.Form):
    text_input = forms.CharField(max_length=255)
    bool_input = forms.BooleanField(required=False)
    int_input = forms.IntegerField(min_value=0,max_value=99)
    email_input = forms.EmailField()

### src/forms_test_recap/views.py

In [ ]:
from django.shortcuts import render

from .forms import RecapForm

def home(request):
    form = RecapForm()
    return render(request, "forms.html", {"form": form}

## Obtener data de un Django form

### src/forms_test_recap/views.py

In [ ]:
from django.shortcuts import render

from .forms import RecapForm

def home(request):
    form = RecapForm(request.POST or None)
    if form.is_valid():
        print(form.data)
    return render(request, "forms.html", {"form": form})

### Validaciones básicas en un Form

### src/forms_test_recap/forms.py

In [ ]:
from django import forms

class RecapForm(forms.Form):
    text_input = forms.CharField(max_length=255)
    bool_input = forms.BooleanField(required=False)
    int_input = forms.IntegerField(min_value=0,max_value=99)
    email_input = forms.EmailField()

    def clean_text(self, *args, **kwargs):
        raw_text = self.cleaned_data.get("text_input")
        if len(raw_text) < 10:
            raise forms.ValidateError("Text must be at least 10 characters long")
        return text_input

    def clean_int(self, *args, **kwargs):
        raw_int = self.cleaned_data.get("int_input")
        if raw_int < 18:
            raise forms.ValidateError("Value must be at least 18")
        return int_input

## Data inicial en un Form

In [ ]:
from django.shortcuts import render

from .forms import RecapForm

def home(request):
    initial_data = {
        "text_input": "initial",
        "bool_input": True,
        "int_input": 18,
        "email_input": "email@email.com",
    }
    form = RecapForm(request.POST or None, initial=initial_data)
    if form.is_valid():
        print(form.data)
    return render(request, "forms.html", {"form": form})

### src/forms_test_recap/forms.py

In [ ]:
from django import forms

class RecapForm(forms.Form):
    text_input = forms.CharField(initial="no sobreescrito")
    bool_input = forms.BooleanField(required=False, initial=False)
    int_input = forms.IntegerField(initial=99)
    email_input = forms.EmailField(initial="sobre@escrito.com")

    def clean_text_input(self):
        raw_text = self.cleaned_data.get("text_input")
        if len(raw_text) < 10:
            print(raw_text)
            raise forms.ValidationError("Text must be at least 10 characters long")
        return raw_text

    def clean_int_input(self):
        raw_int = self.cleaned_data.get("int_input")
        if raw_int < 18:
            raise forms.ValidationError("Value must be at least 18")
        return raw_int

## Etiqueta y Widgets de Forms

In [ ]:
### src/forms_test_recap/forms.py

In [ ]:
from django import forms

YEARS = [x for x in range(1950, 2030)]

OPTIONS = [
    ("oA", "Opción A"),
    ("oB", "Opción B"),
    ("oC", "Opción C"),
]

class RecapForm(forms.Form):
    date_input = forms.DateField(widget=forms.SelectDateWidget(years=YEARS))
    text_input = forms.CharField(
        initial="no sobreescrito",
        label="Descripción del producto",
        widget=forms.Textarea(attrs={"rows": 3, "cols": 42})
    )
    bool_input = forms.BooleanField(required=False, initial=False)
    int_input = forms.IntegerField(initial=99)
    email_input = forms.EmailField(initial="sobre@escrito.com")
    select_input = forms.CharField(label="Selecciona una opción", widget=forms.Select(choices=OPTIONS))
    radio_input = forms.CharField(label="Pick one!", widget=forms.RadioSelect(choices=OPTIONS))
    checkbox_input = forms.CharField(label="Cuáles aplican", widget=forms.CheckboxSelectMultiple(choices=OPTIONS))

    def clean_text_input(self):
        raw_text = self.cleaned_data.get("text_input")
        if len(raw_text) < 30:
            print(raw_text)
            raise forms.ValidationError("Text must be at least 30 characters long")
        return raw_text

    def clean_int_input(self):
        raw_int = self.cleaned_data.get("int_input")
        if raw_int < 18:
            raise forms.ValidationError("Value must be at least 18")
        return raw_int

## Model Form

### src/forms_test_recap/forms.py

In [ ]:
from .models import Product

...

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = Product
        fields =  [
            "title",
            "slug",
            "price",
        ]
        exclude = []

### src/forms_test_recap/models.py

In [ ]:
from django.conf import settings
from django.db import models

class Product(models.Model):
    title = models.CharField(max_length=120)
    slug = models.SlugField(unique=True)
    price = models.FloatField()

In [ ]:
python manage.py makemigrations
python manage.py migrate

### src/forms_test_recap/admin.py

In [ ]:
...

from .models import RecapProduct

admin.site.register(RecapProduct)

### src/forms_test_recap/views.py
- actulizar initial_data
- invocar form.save

In [ ]:
from django.shortcuts import render

from .forms import RecapModelForm

def home(request):
    initial_data = {
        "title": "Product's title",
        "slug": "this-is-a-slug",
        "price": "9.99",
    }
    form = RecapModelForm(request.POST or None, initial=initial_data)
    if form.is_valid():
        form.save()
    return render(request, "forms.html", {"form": form})

## Mensajes de error personalizados

### src/forms_test_recap/urls.py

In [ ]:
path("

## src/forms_test_recap/forms.py

In [ ]:
from django import forms

from .models import RecapProduct

class RecapProductModelForm(forms.ModelForm):
    class Meta:
        model = RecapProduct
        fields = [
            "title",
            "slug",
            "price",
        ]
        labels = {
            "title": "Etiqueta Título",
            "slug": "Etiqueta Slug",
            "price": "Etiqueta Price",
        }
        exclude = []
        
    def clean_title(self):
        cleaned_title = self.cleaned_data.get("title")
        if len(cleaned_title) < 3:
            raise forms.ValidationError("Title is too short.")
        return cleaned_title
    
    def clean_slug(self):
        cleaned_slug = self.cleaned_data.get("slug")
        if len(cleaned_slug) < 3:
            raise forms.ValidationError("Slug is too short.")
        if ("recap" not in cleaned_slug):
            raise forms.ValidationError("Slug must contain 'recap'.")
        return cleaned_slug

## Django Formsets

### src/templates/base.html

In [ ]:
<body>
    <div class="container">

        {% block content %}
        {% endblock %}

    </div>
</body>

### src/templates/formset_view.html

In [ ]:
{% extends "base.html" %}

{% block content %}

<h1>Formset View</h1>

<form action="" method="POST">
    {% csrf_token %}

    {{ formset.management_form }}
    {% for form in formset %}
        <div>
            {{ form.as_p }}
            <br/>
        </div>
    {% endfor %}

    <input type="submit" value="Guardar">
</form>

{% endblock %}

### src/forms_test_recap/views.py

In [ ]:
from django.forms import formset_factory, modelformset_factory
from django.shortcuts import render

from .forms import RecapForm, RecapProductModelForm

def create_recap_product(request):
    RecapFormSet = formset_factory(RecapForm, extra=3)
    formset = RecapFormSet(request.POST or None)
    for form in formset:
        print(form.data)
    context = {
        "formset": formset
    }
    return render(request, "formset_view.html", context)


## Django ModelFormsets

In [ ]:
from django.forms import formset_factory, modelformset_factory
from django.shortcuts import render

from .forms import RecapForm, RecapProductModelForm
from .models import RecapProduct

def create_recap_product(request):
    RecapModelFormSet = modelformset_factory(RecapProduct, form=RecapProductModelForm)
    formset = RecapModelFormSet(request.POST or None, queryset=RecapProduct.objects.all())
    
    print("formset.data")
    print(formset.data)

    print("formset.errors")
    print(formset.errors)
    
    if formset.is_valid():
        print("ModelFormset es válido")
        formset.save()
    context = {
        "formset": formset
    }
    return render(request, "formset_view.html", context)

# Actividad de módulo

### src/forms_test_recap/models.py

In [ ]:
from django.db import models

class RecapProduct(models.Model):
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    
class User(models.Model):
    username = models.CharField(max_length=150, unique=True)
    email = models.EmailField(unique=True)
    password = models.CharField(max_length=128)